# EP1: Cálculo do Conjunto de Mandelbrot em Paralelo com Pthreads e OpenMP

## Entrega do EP1

Preencha o nome dos 5 membros do seu grupo na tabela abaixo:

| Nome | NUSP |
|------|------|
| André Akira Hayashi | 9293011 |
| Mateus Agostinho dos Anjos | 9298191 |
| Renan Tiago dos Santos Silva | 9793606 |
| Rodrigo Toscano Ney | 11361219 |
| Bruno Picoli Romano | 6402851 |

In [1]:
] up

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching: [=======================>                 ]  

Fetching: [========================================>]  100.0 %

  Installed FixedPointNumbers ─ v0.8.2
   Updating `~/.julia/environments/v1.4/Project.toml`
  [8314cec4] ↑ PGFPlotsX v1.2.6 ⇒ v1.2.8
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [53c48c17] ↑ FixedPointNumbers v0.8.1 ⇒ v0.8.2
  [8314cec4] ↑ PGFPlotsX v1.2.6 ⇒ v1.2.8


In [2]:
import Pkg;
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add(Pkg.PackageSpec(;name="PGFPlotsX", version="1.2.6"))
Pkg.build("PGFPlotsX")
Pkg.add("Plots")
Pkg.add("Query")
Pkg.add("StatsPlots")

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
  [8314cec4] ↓ PGFPlotsX v1.2.8 ⇒ v1.2.6
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [8314cec4] ↓ PGFPlotsX v1.2.8 ⇒ v1.2.6
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
 

In [3]:
] st

Status `~/.julia/environments/v1.4/Project.toml`
  [336ed68f] CSV v0.7.3
  [a93c6f00] DataFrames v0.21.4
  [7073ff75] IJulia v1.21.2
  [8314cec4] PGFPlotsX v1.2.6
  [91a5bcdd] Plots v1.5.4
  [1a8c2f83] Query v0.12.2
  [f3b207a7] StatsPlots v0.14.6


Função que lê um arquivo <em>file</em> e retorna ele em linhas.

In [4]:
function read_file(file)
    f = open(file);
    return lines = readlines(f)
end

read_file (generic function with 1 method)

## SEQ, PTH, OMP
Função que lê as linhas de um arquivo e retorna ele em forma de um Dataframe.

In [13]:
using DataFrames
using Printf

function measure_mandelbrot(iteration, threads, start, lines)
    info = Vector(undef, 10)
    end_line = start + 9
    for n in start:end_line
        i = ((n-start) + 1)
        new = lstrip(lines[n])
        sp = split(new, " ")
        info[i] = sp[1]
    end
    return DataFrame(task_clock = parse(Float64,info[1]),
        context_switches = parse(Float64,info[2]),
        cpu_migrations = parse(Float64,info[3]),
        page_faults = parse(Int,info[4]),
        time_elapsed = parse(Float64,info[10]),
        iteration = iteration,
        threads = threads)
end

measure_mandelbrot (generic function with 1 method)

## CUDA

In [5]:
using DataFrames
using Printf

function measure_mandelbrot(iteration, threads, start, lines)
    info = Vector(undef, 13)
    end_line = start + 12
    for n in start:end_line
        i = ((n-start) + 1)
        new = lstrip(lines[n])
        sp = split(new, " ")
        info[i] = sp[1]
    end
    return DataFrame(task_clock = parse(Float64,info[1]),
        context_switches = parse(Float64,info[2]),
        cpu_migrations = parse(Float64,info[3]),
        page_faults = parse(Int,info[4]),
        time_elapsed = parse(Float64,info[13]),
        iteration = iteration,
        threads = threads)
end

measure_mandelbrot (generic function with 1 method)

## SEQ, PTH, OMP
Função que gera um Dataframe dada o tamanho da imagem, o número de threads e as linhas do arquivo.

In [6]:
function make_Dataframe(iteration, threads, lines)
    results = DataFrame(task_clock = Float64[],
        context_switches = Float64[],
        cpu_migrations = Float64[],
        page_faults = Int[],
        time_elapsed = Float64[],
        iteration = Int[],
        threads = Int[])
    start = 4
    for s in iteration
        for t in threads
            append!(results,
                measure_mandelbrot(s, t, start, lines)) 
            start = start + 14
        end
    end
    return results
end

make_Dataframe (generic function with 1 method)

## CUDA

In [6]:
function make_Dataframe(iteration, threads, lines)
    results = DataFrame(task_clock = Float64[],
        context_switches = Float64[],
        cpu_migrations = Float64[],
        page_faults = Int[],
        time_elapsed = Float64[],
        iteration = Int[],
        threads = Int[])
    start = 4
    for s in iteration
        for t in threads
            append!(results,
                measure_mandelbrot(s, t, start, lines)) 
            start = start + 17
        end
    end
    return results
end

make_Dataframe (generic function with 1 method)

Função que salva um Dataframe em .csv com o nome <em>filename</em>.

In [7]:
using CSV

function save_csv_results(DataFrame, filename)
    fname = string(filename, ".csv")
    CSV.write(fname, DataFrame)
end
function read_csv_results(filename)
    copy = CSV.read(filename)
    return copy
end

read_csv_results (generic function with 1 method)

## Exercício 1: Apresentação e Análise de Medições para o Programa Sequencial

Os arquivos lidos não continham "," e os **elementos** <em>task-clock</em>, <em>context-switches</em>, <em>cpu-migrations</em>, <em>page-faults</em>, <em>cycles</em>, <em>instuctions</em>, <em>branches</em>, <em>branch-misses</em> e <em>time elpsed</em>, para todos as versões.

Aqui é retirado a informação dos arquivos de log gerado pelo algoritmo sequencial de Mandelbrot, e é colocado em seus respectivos Dataframes. Altera-se os valores de <em>MAX_SIZE</em> e <em>MAX_NUM_THREAD</em> para alterar respectivamente o tamanho da imagem e o número de threads, ambos os valores são na base $2^x$.

Como para as análises abaixo utilizamos as versões com e sem I/O deve executar ambos o arquivos: **mandelbrot_seq** e **mandelbrot_seq_noio**. Ou basta executar o Makefile com o run_measurement.sh, disponível com o arquivo.

In [8]:
using DataFrames, Query, StatsPlots, Statistics
function parse_results_threads(results)
    parsed_results = results |>
                    @groupby(_.threads) |>
                    @map({threads = key(_),
                          mean_time = mean(_.time_elapsed),
                          ci_time = 1.96 * std(_.time_elapsed)}) |>
                    DataFrame
    
    return parsed_results
end

function parse_results_iteration(results)
    parsed_results = results |>
                    @groupby(_.iteration) |>
                    @map({iteration = key(_),
                          mean_time = mean(_.time_elapsed),
                          ci_time = 1.96 * std(_.time_elapsed)}) |>
                    DataFrame
    
    return parsed_results
end

┌ Info: Precompiling StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1260


parse_results_iteration (generic function with 1 method)

In [10]:
MAX_SIZE = 8
MAX_NUM_THREAD = 5
s = [x for x in 1:MAX_SIZE]
threads = [2 ^ x for x in 0:MAX_NUM_THREAD]

lines_cuda_triple_spiral = read_file("results/mandelbrot_cuda_noio/triple_spiral_v2.log")
results_cuda_triple_spiral = make_Dataframe(s, threads, lines_cuda_triple_spiral)

save_csv_results(results_cuda_triple_spiral, "results_cuda_triple_spiral")
cuda_triple_spiral = read_csv_results("results_cuda_triple_spiral.csv")
parsed_threads_cuda_triple_spiral = (parse_results_threads(cuda_triple_spiral))
parsed_iteration_cuda_triple_spiral = (parse_results_iteration(cuda_triple_spiral))

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /home/akira/.julia/packages/CSV/W9RT2/src/CSV.jl:40


,iteration,mean_time,ci_time
,Int64,Float64,Float64
1,1,0.710512,0.0334223
2,2,0.710862,0.0135739
3,3,0.719982,0.02325
4,4,0.71256,0.0261993
5,5,0.70973,0.0275163
6,6,0.720868,0.0186025
7,7,0.72845,0.0190411
8,8,0.727308,0.0222488


In [11]:
MAX_SIZE = 8
MAX_NUM_THREAD = 6
s = [x for x in 1:MAX_SIZE]
threads = [x for x in 2:MAX_NUM_THREAD]

lines_ompi_triple_spiral = read_file("results/mandelbrot_ompi_noio/triple_spiral.log")
results_ompi_triple_spiral = make_Dataframe(s, threads, lines_ompi_triple_spiral)

save_csv_results(results_ompi_triple_spiral, "results_ompi_triple_spiral")
ompi_triple_spiral = read_csv_results("results_ompi_triple_spiral.csv")
parsed_threads_ompi_triple_spiral = (parse_results_threads(ompi_triple_spiral))
parsed_iteration_ompi_triple_spiral = (parse_results_iteration(ompi_triple_spiral))

┌ Warning: `CSV.read(input; kw...)` is deprecated in favor of `DataFrame!(CSV.File(input; kw...))`
└ @ CSV /home/akira/.julia/packages/CSV/W9RT2/src/CSV.jl:40


,iteration,mean_time,ci_time
,Int64,Float64,Float64
1,1,9.83536,11.4659
2,2,9.84052,11.5043
3,3,9.81962,11.498
4,4,9.82162,11.4967
5,5,9.82236,11.4868
6,6,9.83923,11.4725
7,7,9.82042,11.5048
8,8,9.83832,11.4662


In [57]:
MAX_SIZE = 8
MAX_NUM_THREAD = 0
s = [x for x in 1:MAX_SIZE]
threads = [2 ^ x for x in 0:MAX_NUM_THREAD]

lines_seq_triple_spiral_noio = read_file("results/mandelbrot_seq_noio/triple_spiral.log")
results_seq_triple_spiral_noio = make_Dataframe(s, threads, lines_seq_triple_spiral_noio)

save_csv_results(results_seq_triple_spiral_noio, "results_seq_triple_spiral_noio")
seq_triple_spiral = read_csv_results("results_seq_triple_spiral_noio.csv")
parsed_threads_seq_triple_spiral = (parse_results_threads(seq_triple_spiral))
parsed_iteration_seq_triple_spiral = (parse_results_iteration(seq_triple_spiral))

,iteration,mean_time,ci_time
,Int64,Float64,Float64
1,1,22.1969,NaN
2,2,22.3583,NaN
3,3,22.1904,NaN
4,4,22.1909,NaN
5,5,22.1964,NaN
6,6,22.1899,NaN
7,7,22.1931,NaN
8,8,22.1983,NaN


In [58]:
MAX_SIZE = 8
MAX_NUM_THREAD = 5
s = [x for x in 1:MAX_SIZE]
threads = [2 ^ x for x in 0:MAX_NUM_THREAD]

lines_pth_triple_spiral_noio = read_file("results/mandelbrot_pth_noio/triple_spiral.log")
results_pth_triple_spiral_noio = make_Dataframe(s, threads, lines_pth_triple_spiral_noio)
save_csv_results(results_pth_triple_spiral_noio, "results_pth_triple_spiral_noio")
pth_triple_spiral = read_csv_results("results_pth_triple_spiral_noio.csv")
parsed_threads_pth_triple_spiral = (parse_results_threads(pth_triple_spiral))
parsed_iteration_pth_triple_spiral = (parse_results_iteration(pth_triple_spiral))

lines_omp_triple_spiral_noio = read_file("results/mandelbrot_omp_noio/triple_spiral.log")
results_omp_triple_spiral_noio = make_Dataframe(s, threads, lines_omp_triple_spiral_noio)
save_csv_results(results_omp_triple_spiral_noio, "results_omp_triple_spiral_noio")
omp_triple_spiral = read_csv_results("results_omp_triple_spiral_noio.csv")
parsed_threads_omp_triple_spiral = (parse_results_threads(omp_triple_spiral))
parsed_iteration_omp_triple_spiral = (parse_results_iteration(omp_triple_spiral))

,iteration,mean_time,ci_time
,Int64,Float64,Float64
1,1,10.2685,12.2909
2,2,10.2721,12.2465
3,3,10.2795,12.2408
4,4,10.2927,12.2121
5,5,10.3164,12.2111
6,6,10.3156,12.177
7,7,10.3254,12.1858
8,8,10.3294,12.1734
